<a href="https://colab.research.google.com/github/O-Kpy/Kaggle/blob/main/tabular_playground_series_jun_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import skew

In [ ]:
submit = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/sample_submission.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/test.csv')
train = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/train.csv')
train.shape, test.shape, submit.shape

((200000, 77), (100000, 76), (100000, 10))

In [ ]:
train.target.value_counts().sort_index(ascending=False).plot.bar()

In [ ]:
data=pd.concat([train, test], axis=0)

In [ ]:
data.isnull().sum().sort_values(ascending=False)
# Null값 없고

In [ ]:
data.skew().sort_values(ascending=False)
# 스케일링 해야하고

In [ ]:
for i in train.drop(columns=['id', 'target']).columns:
    if train[i].skew() > 0.5:
        train[i] = np.log1p(train[i]+1)
        
for i in test.drop(columns=['id']).columns:
    if test[i].skew() > 0.5:
        test[i] = np.log1p(test[i]+1)

In [ ]:
np.array(train['feature_0'])

array([0, 0, 0, ..., 1, 0, 5])

In [ ]:
PowerTransformer().fit_transform(np.array(train['feature_0']).reshape(1, -1))

KeyboardInterrupt: 

In [ ]:
# boxcox로 Normalize하기
from sklearn.preprocessing import PowerTransformer

for i in train.drop(columns=['id', 'target']).columns:
    train[i] = PowerTransformer().fit_transform(train[i])

ValueError: Expected 2D array, got 1D array instead:
array=[0.69314718 0.69314718 0.69314718 ... 1.09861229 0.69314718 1.94591015].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
train_agg = train.drop(columns='target').groupby('id').agg(['median'])
train_agg_col = []

for i in train_agg.columns.levels[0]:
    for j in train_agg.columns.levels[1]:
        train_agg_col.append(f'{i}-{j}')
        
train_agg.columns = train_agg_col

test_agg = test.groupby('id').agg(['median'])
test_agg_col = []

for i in test_agg.columns.levels[0]:
    for j in test_agg.columns.levels[1]:
        test_agg_col.append(f'{i}-{j}')
        
test_agg.columns = test_agg_col

In [ ]:
corr_matrix = train_agg.corr()

upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

to_drop = [c for c in upper.columns if any(abs(upper[c]) > 0.9)]

print(f'There are {len(train_agg.columns)} columns but {len(to_drop)} corr columns to remove')

There are 75 columns but 0 corr columns to remove


In [ ]:
train_agg.drop(columns=to_drop)
train = train.merge(train_agg, how='left', on='id')

test_agg.drop(columns=to_drop)
test = test.merge(test_agg, how='left', on='id')

# 그냥 모델링 하기 다음에 피쳐 엔지니어링

In [ ]:
from sklearn.metrics import log_loss, make_scorer
import catboost
import lightgbm
import xgboost
import optuna
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate, KFold, StratifiedKFold, StratifiedShuffleSplit, train_test_split
from sklearn.preprocessing import LabelEncoder, robust_scale, MinMaxScaler

In [ ]:
x_train = train.drop(columns=['target', 'id'])
y_train = train['target']
x_test = test.drop(columns=['id'])

In [ ]:
x_train = data.loc[data['target'].notnull()].drop(columns=['id', 'target'])
y_train = data.loc[data['target'].notnull()]['target']
x_test = data.loc[data['target'].isnull()].drop(columns=['id', 'target'])

In [ ]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_train = pd.Series(y_train)

In [ ]:
x_train_col = x_train.columns
x_test_col = x_test.columns

x_train = pd.DataFrame(robust_scale(x_train), columns=x_train_col)
x_test = pd.DataFrame(robust_scale(x_test), columns=x_test_col)

In [ ]:
# lgbm 모델
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
preds = np.zeros((len(x_test)))
for i, (train_index, valid_index) in enumerate(skf.split(x_train, y_train)):
  X_train, X_valid = x_train.iloc[train_index], x_train.iloc[valid_index]
  Y_train, Y_valid = y_train.iloc[train_index], y_train.iloc[valid_index]

  model = lightgbm.LGBMClassifier(n_estimators=3000, learning_rate=0.05, objective='multiclass', n_jobs=-1)
  model.fit(X_train, Y_train, eval_set=[(X_train, Y_train), (X_valid, Y_valid)], verbose=100, early_stopping_rounds=70, eval_metric='multi_logloss')
  preds = model.predict_proba(x_test)

In [ ]:
def objective(trial,data=x_train,target=y_train):
    
    X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2,random_state=42)
    params = {'iterations':trial.suggest_int("iterations", 2000, 25000),
             'loss_function':'MultiClass',
              'task_type':"GPU",
              'eval_metric':'MultiClass',
              'leaf_estimation_method':'Newton',
              'bootstrap_type': 'Bernoulli',
              'learning_rate' : trial.suggest_uniform('learning_rate',0.01,0.5),
              'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
              'subsample': trial.suggest_uniform('subsample',0,1),
              'random_strength': trial.suggest_uniform('random_strength',10,50),
              'depth': trial.suggest_int('depth',1,10),
              'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,50),
              'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,15),
              'grow_policy' : 'Lossguide'
               }
    model = catboost.CatBoostClassifier(**params)  
    model.fit(X_train,y_train,eval_set=[(X_test,y_test)],early_stopping_rounds=100,verbose=False)
        
    y_preds = model.predict_proba(X_test)


    log_loss_multi = log_loss(y_test, y_preds)
    
    return log_loss_multi

In [ ]:
OPTUNA_OPTIMIZATION = True

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)
print('Number of finished trials:', len(study.trials))
print('Best trial: score {}, params {}'.format(study.best_trial.value, study.best_trial.params))

[I 2021-06-26 08:12:29,088] A new study created in memory with name: no-name-cef43c19-da84-416e-a4ec-b6ca835aa262
[I 2021-06-26 08:13:34,688] Trial 0 finished with value: 1.7591363047184783 and parameters: {'iterations': 6973, 'learning_rate': 0.3941153419269652, 'reg_lambda': 19.557147522026, 'subsample': 0.17712609302918558, 'random_strength': 34.71799720978931, 'depth': 12, 'min_data_in_leaf': 2, 'leaf_estimation_iterations': 12}. Best is trial 0 with value: 1.7591363047184783.
[I 2021-06-26 08:13:40,914] Trial 1 finished with value: 1.7528753888232358 and parameters: {'iterations': 15209, 'learning_rate': 0.24341264300966542, 'reg_lambda': 23.656405417398528, 'subsample': 0.010689323312411902, 'random_strength': 15.521222938234946, 'depth': 11, 'min_data_in_leaf': 39, 'leaf_estimation_iterations': 3}. Best is trial 1 with value: 1.7528753888232358.
[I 2021-06-26 08:13:48,942] Trial 2 finished with value: 1.7507235773538095 and parameters: {'iterations': 15437, 'learning_rate': 0.14

Number of finished trials: 100
Best trial: score 1.746029926356897, params {'iterations': 3150, 'learning_rate': 0.01722922366960469, 'reg_lambda': 97.87444502097921, 'subsample': 0.33141204073461017, 'random_strength': 26.6396026843073, 'depth': 8, 'min_data_in_leaf': 12, 'leaf_estimation_iterations': 1}


In [ ]:
# catboost모델 제일 성능이 잘 나옴(NN빼고)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
preds_cat = np.zeros((x_test.shape[0], y_train.nunique()))
params_cat = {'iterations':study.best_params['iterations'],
             'loss_function':'MultiClass',
              'task_type':"GPU",
              'eval_metric':'MultiClass',
              'leaf_estimation_method':'Newton',
              'bootstrap_type': 'Bernoulli',
              'learning_rate' : study.best_params['learning_rate'],
              'reg_lambda': study.best_params['reg_lambda'],
              'subsample': study.best_params['subsample'],
              'random_strength': study.best_params['random_strength'],
              'depth': study.best_params['depth'],
              'min_data_in_leaf': study.best_params['min_data_in_leaf'],
              'leaf_estimation_iterations': study.best_params['leaf_estimation_iterations'],
              'grow_policy' : 'Lossguide'
               }

for i, (train_index, valid_index) in enumerate(skf.split(x_train, y_train)):
    X_train, X_valid = x_train.iloc[train_index], x_train.iloc[valid_index]
    Y_train, Y_valid = y_train.iloc[train_index], y_train.iloc[valid_index]
    
    model = catboost.CatBoostClassifier(**params_cat)
    model.fit(X_train, Y_train, verbose=100, early_stopping_rounds=70, eval_set=(X_valid, Y_valid))
    preds_cat += model.predict_proba(x_test)/skf.n_splits


0:	learn: 2.1834006	test: 2.1835584	best: 2.1835584 (0)	total: 22.6ms	remaining: 1m 11s
100:	learn: 1.7936150	test: 1.8017998	best: 1.8017998 (100)	total: 2.39s	remaining: 1m 12s
200:	learn: 1.7528883	test: 1.7653191	best: 1.7653191 (200)	total: 4.4s	remaining: 1m 4s
300:	learn: 1.7420348	test: 1.7577615	best: 1.7577615 (300)	total: 6.38s	remaining: 1m
400:	learn: 1.7364549	test: 1.7550311	best: 1.7550311 (400)	total: 8.32s	remaining: 57s
500:	learn: 1.7324424	test: 1.7536818	best: 1.7536818 (500)	total: 10.3s	remaining: 54.2s
600:	learn: 1.7291414	test: 1.7527713	best: 1.7527713 (600)	total: 12.2s	remaining: 51.7s
700:	learn: 1.7262984	test: 1.7521609	best: 1.7521609 (700)	total: 14.3s	remaining: 50s
800:	learn: 1.7237611	test: 1.7517072	best: 1.7517072 (800)	total: 16.2s	remaining: 47.5s
900:	learn: 1.7214533	test: 1.7513777	best: 1.7513777 (900)	total: 18s	remaining: 45s
1000:	learn: 1.7193082	test: 1.7511074	best: 1.7511074 (1000)	total: 19.9s	remaining: 42.7s
1100:	learn: 1.717217

In [ ]:
model_cat.select_features(
                X_train,
                Y_train,
                eval_set=[(X_train, Y_train), (X_valid, Y_valid)],
                features_for_select=list(len(X_train.columns)),
                num_features_to_select=150,
                algorithm='RecursiveByLossFunctionChange',
                steps=None,
                shap_calc_type=None,
                train_final_model=False,
                verbose=None,
                logging_level=None,
                plot=True)

In [ ]:
from catboost.utils import get_gpu_device_count
get_gpu_device_count()

In [ ]:
# xgboost ==> 너무 느림
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
preds_xgb = np.zeros((len(x_test)))
for i, (train_index, valid_index) in enumerate(skf.split(x_train, y_train)):
  X_train, X_valid = x_train.iloc[train_index], x_train.iloc[valid_index]
  Y_train, Y_valid = y_train.iloc[train_index], y_train.iloc[valid_index]

  model_xgb = xgboost.XGBClassifier(n_estimators=3000, learning_rate=0.022, objective='multiclass', tree_method='gpu_hist')
  model_xgb.fit(X_train, Y_train, eval_set=[(X_train, Y_train), (X_valid, Y_valid)], verbose=100, early_stopping_rounds=70, eval_metric='mlogloss')
  preds_xgb = model_xgb.predict_proba(x_test)

[0]	validation_0-mlogloss:2.18487	validation_1-mlogloss:2.18537
[100]	validation_0-mlogloss:1.77220	validation_1-mlogloss:1.80678
[200]	validation_0-mlogloss:1.69948	validation_1-mlogloss:1.76371
[300]	validation_0-mlogloss:1.66592	validation_1-mlogloss:1.75669
[400]	validation_0-mlogloss:1.64265	validation_1-mlogloss:1.75584
[450]	validation_0-mlogloss:1.63327	validation_1-mlogloss:1.75592
[0]	validation_0-mlogloss:2.18492	validation_1-mlogloss:2.18520
[100]	validation_0-mlogloss:1.77369	validation_1-mlogloss:1.80184
[200]	validation_0-mlogloss:1.70139	validation_1-mlogloss:1.75738
[300]	validation_0-mlogloss:1.66809	validation_1-mlogloss:1.74991
[400]	validation_0-mlogloss:1.64513	validation_1-mlogloss:1.74853
[500]	validation_0-mlogloss:1.62745	validation_1-mlogloss:1.74842
[551]	validation_0-mlogloss:1.61918	validation_1-mlogloss:1.74856
[0]	validation_0-mlogloss:2.18492	validation_1-mlogloss:2.18524
[100]	validation_0-mlogloss:1.77357	validation_1-mlogloss:1.80286
[200]	validation

In [ ]:
submit = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/sample_submission.csv')
submit = submit.set_index('id')

In [ ]:
for i in range(9):
    submit.iloc[:,i] = preds_cat[:,i]

In [ ]:
submit.to_csv('submit16.csv')

# NN 파이토치 필사본

In [ ]:
import os
import sys
import datetime

import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import optuna
from optuna.trial import TrialState

In [ ]:
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.3, random_state=42)

for train_index, val_index in sss.split(x_train, y_train):
    print(f'train size:{len(train_index)}')
    print(f'val size:{len(val_index)}')
    X_train, X_val = x_train.iloc[train_index], x_train.iloc[val_index]
    Y_train, Y_val = y_train.iloc[train_index], y_train.iloc[val_index]

# Optuna

In [ ]:
pip install optuna

In [ ]:
pip install skorch

In [ ]:
import optuna
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
CLASSES = y_train.nunique()
NUM_FEATURES = x_train.shape[1]
DEVICE = torch.device("cpu" if not torch.cuda.is_available() else "cuda")

BATCHSIZE = 1000
EPOCHS = 10
N_TRAIN_EXAMPLES = BATCHSIZE * 14
N_VALID_EXAMPLES = BATCHSIZE * 6

criterion = nn.CrossEntropyLoss()

def define_model(trial):
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    n_layers = trial.suggest_int("n_layers", 1, 10)
    layers = []

    in_features = NUM_FEATURES
    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), y_train.nunique(), 1024)
        layers.append(nn.Linear(in_features, out_features, bias=False))
        layers.append(nn.ReLU(inplace=True))
        p = trial.suggest_float("dropout_l{}".format(i), 0.1, 0.5)
        layers.append(nn.Dropout(p))
        layers.append(nn.BatchNorm1d(out_features))

        in_features = out_features
        
    layers.append(nn.Linear(in_features, CLASSES))

    return nn.Sequential(*layers)

def get_data(X_train, y_train, X_val, y_val):
    train = TensorDataset(torch.Tensor(np.array(X_train)), torch.Tensor(np.array(y_train)))
    train_loader = DataLoader(train, batch_size = 1000, shuffle = True, num_workers=4)

    val = TensorDataset(torch.Tensor(np.array(X_val)), torch.Tensor(np.array(y_val)))
    val_loader = DataLoader(val, batch_size = 1000, shuffle = True, num_workers=4)

    return train_loader, val_loader

def objective(trial):
    model = define_model(trial).to(DEVICE)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", 'AdamW', "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 0.001, 0.5)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr, weight_decay)
    lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer)
    train_loader, valid_loader = get_data(X_train, Y_train, X_val, Y_val)
    
    for epoch in range(EPOCHS):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)

            optimizer.zero_grad()
            output = model(data)
            
            loss = criterion(output, target.long())
            loss.backward()
            optimizer.step()
            lr_scheduler.step(epoch + batch_idx / len(train_loader))

        # Validation of the model.
        model.eval()
        val_loss = []
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(valid_loader):
                data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)
                output = model(data)
                
                loss = criterion(output, target.long())
                val_loss.append(loss.item())
        
        avg_val_loss = np.mean(val_loss)

        trial.report(avg_val_loss, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    
    return avg_val_loss

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

In [ ]:
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print('Study statistics:')
print(' Number of finished trials:', len(study.trials))
print(' Number of pruned trials:', len(pruned_trials))
print(' Number of complete trials:', len(complete_trials))

print('Best trial:')
trial = study.best_trial

print(' Value:', trial.value)
print(' Params:')
for key, value in trial.params.items():
    print(f'{key}:{value}')

In [ ]:
optuna.visualization.plot_intermediate_values(study).show()

# 최적화된 파라미터를 가지고 모델링

In [ ]:
class classification_model(nn.Module):
    def __init__(self, n_in, n_out, layers, p=None):
        super(classification_model, self).__init__()
        
        all_layers=[]
        self.n_in = n_in
        self.n_out = n_out
        
        for i in range(len(layers)):
            all_layers.append(nn.Linear(self.n_in, layers[i], bias=False))
            all_layers.append(nn.ReLU(inplace=True))
            if p:
                all_layers.append(nn.Dropout(p[i]))
            all_layers.append(nn.BatchNorm1d(layers[i]))
            
            self.n_in = layers[i]
            
        all_layers.append(nn.Linear(layers[-1], self.n_out))
        
        self.layers=nn.Sequential(*all_layers)
        
    def forward(self, x):
        x = self.layers(x)
        return x
    
    net_name='classification_model'

In [ ]:
batch_size = 1000

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=0)

for train_index, val_index in sss.split(x_train, y_train):    
    print("TRAIN:\t", train_index, "Size:\t", len(train_index))
    print("VAL:\t", val_index, "Size:\t", len(val_index))
    X_train, X_val = x_train.iloc[train_index], x_train.iloc[val_index]
    Y_train, Y_val = y_train.iloc[train_index], y_train.iloc[val_index]

train = TensorDataset(torch.Tensor(np.array(X_train)), torch.Tensor(np.array(Y_train)))
train_loader = DataLoader(train, batch_size = batch_size, shuffle = True)

val = TensorDataset(torch.Tensor(np.array(X_val)), torch.Tensor(np.array(Y_val)))
val_loader = DataLoader(val, batch_size = batch_size, shuffle = True)

phases = ["train", "val"]
loaders = {"train": train_loader, "val": val_loader}

In [ ]:
"""
Best trial:
 Value: 1.7687533100446065
 Params:
n_layers:1
n_units_l0:880
dropout_l0:0.2116281048320166
optimizer:RMSprop
lr:0.0009784821223970022
"""

layers_=[]
dropout_=[]
l_rate=trial.params['lr']

for i in range(trial.params['n_layers']):
    layers_.append(trial.params[f'n_units_l{i}'])
    dropout_.append(trial.params[f'dropout_l{i}'])
    
device = torch.device('cpu' if not torch.cuda.is_available() else 'cuda')

model = classification_model(x_train.shape[1], y_train.nunique(), layers_, p=dropout_)
model.to(device)

criterion = nn.CrossEntropyLoss()
if trial.params['optimizer'] == 'RMSprop':
    optimizer = optim.RMSprop(model.parameters(), lr=l_rate)
elif trial.params['optimizer'] == 'Adam':
    optimizer = optim.Adam(model.parameters(), lr=l_rate)
elif trial.params['optimizer'] == 'SGD':
    optimizer = optim.SGD(model.parameters(), lr=l_rate)
    
print(device)
print(model)

In [ ]:
loss_train=[]
loss_valid=[]
best_validation_loss = 1000
best_epoch=1
correct=0
total=0

n_epochs=50

now = datetime.datetime.now()
weights_path = "./output/{:%Y%m%dT%H%M}".format(now)
os.makedirs(weights_path, exist_ok=True)

lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer)
break_st = False

for epoch in range(1, n_epochs+1):
    
    for phase in phases:
        if phase == 'train':
            model.train()
        elif phase == 'val':
            model.eval()
        
        for _, data in enumerate(loaders[phase], 0):
            features, y_true = data[0], data[1]
            features = features.to(device, dtype=torch.float)
            y_true = y_true.to(device, dtype=torch.float)
            
            optimizer.zero_grad()
            
            with torch.set_grad_enabled(phase=='train'):
                y_pred = model(features)
                
                sm = nn.Softmax(dim=1)  # predict_proba
                pred_percentage = sm(y_pred)
                
                if break_st:
                    break
                    
                y_true = y_true.long()
                
                _, preds = torch.max(pred_percentage, 1)
                total += y_true.size(0)
                correct += (preds == y_true).sum().item()
                
                loss = criterion(y_pred, y_true)
                
                if phase == 'val':
                    loss_valid.append(loss.item())
                    
                if phase == 'train':
                    loss_train.append(loss.item())
                    loss.backward()
                    
                    optimizer.step()
                    lr_scheduler.step(epoch / len(train_loader))
        if break_st:
            break
        if phase == 'train':
            mean_train_loss = np.mean(loss_train)
            acc_train = 100*correct/total
            loss_train = []
            correct = 0
        
        if phase == 'val':
            validation_loss = np.mean(loss_valid)
            acc_valid = 100*correct/total
            loss_valid = []
            correct = 0
            total = 0
                
    if break_st:
        break
    if validation_loss < best_validation_loss:
        print('saving weights...')
        best_epoch = epoch
        best_validation_loss = validation_loss
        torch.save(model.state_dict(),
                   os.path.join(weights_path, 'model.pt'),)
            
    print(f'Epoch={epoch}/{n_epochs}\tloss={mean_train_loss:.4f}\tval_loss={validation_loss:.4f}\tacc={acc_train:.4f}\tval_acc={acc_valid:.4f}')
                


In [ ]:
test = TensorDataset(torch.Tensor(np.array(x_test)))
test_loader = DataLoader(test, batch_size=100000, shuffle=False)

In [ ]:
if not weights_path:
    print('Choose weights path')
    sys.exit()

device = torch.device('cpu' if not torch.cuda.is_available() else 'cuda')

model_name = weights_path + '/model.pt'
print(model_name)
state_dict = torch.load(model_name, map_location=device)
model.load_state_dict(state_dict)
model.eval()

for _, data in enumerate(test_loader, 0):
    features = data[0]
    print(features.size())
    features = features.to(device, dtype=torch.float)
    
    with torch.set_grad_enabled(False):
        y_pred = model(features)
        
        sm = nn.Softmax(dim=1)
        pred_percentage = sm(y_pred)
        
        print(pred_percentage.size())

print(pred_percentage.detach().cpu().numpy())
print('Done!')

In [ ]:
submit = pd.read_csv('../input/tabular-playground-series-jun-2021/sample_submission.csv')
pred_array = pred_percentage.detach().cpu().numpy()
submit.loc[:, 'Class_1':'Class_9'] = pred_array
submit = submit.set_index('id')
submit.head()

In [ ]:
submit.to_csv('./output/submission10.csv')

# NN모델을 써야 점수가 올라간다. 
# Pytorch NN모델을 생성하고 제출하자
# feature_select할게 더 있나? 체크